In [6]:
import os

import nltk
import wrds
import pandas as pd

In [2]:
conn = wrds.Connection(wrds_username='denisfench')

Loading library list...
Done


In [5]:
### Get S&P500 Index Membership from CRSP

sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])


### Add Other Company Identifiers from CRSP.MSENAMES
### - You don't need this step if only PERMNO is required
### - This step aims to add TICKER, SHRCD, EXCHCD and etc.

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt,
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

print("The length of the full SP500 list is: ", len(sp500_full))

The length of the full SP500 list is:  985474


In [6]:

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]


### Add Other Company Identifiers from CRSP.MSENAMES

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt,
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]


### Add Compustat Identifiers

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno,
                  linktype, linkprim, linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]

# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', 'linktype', \
                                  'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip',\
                     'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]


### Add CIKs and Link with SEC Index Files using CIK

names = conn.raw_sql(""" select gvkey, cik, sic, naics, gind, gsubind from comp.names """)

# Merge sp500 constituents table with names table
sp500 = pd.merge(sp500ccm, names, on='gvkey',  how='left')
sp500.head(20)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret,cik,sic,naics,gind,gsubind
0,2000-01-31,40416.0,AVON PRODUCTS INC,05430310,11.0,1.0,2844.0,AVP,001920,01,1967-05-18,2015-03-20,-0.035985,0000008868,2844,325620,303020,30302010
1,2000-01-31,44062.0,SPRINGS INDUSTRIES INC,85178310,11.0,1.0,2221.0,SMI,009963,01,1967-06-29,2000-12-11,-0.089202,0000093102,2211,313210,252010,25201020
2,2000-01-31,26403.0,DISNEY WALT CO,25468710,11.0,1.0,4833.0,DIS,003980,01,1976-07-01,2022-03-31,0.241453,0001744489,4888,515120,502020,50202010
3,2000-01-31,60628.0,FEDEX CORP,31428X10,11.0,1.0,4513.0,FDX,004598,01,1980-11-06,2022-03-31,-0.033588,0001048911,4513,492110,203010,20301010
4,2000-01-31,69032.0,MORGAN STANLEY DEAN WITTER & CO,61744644,11.0,1.0,6282.0,MWD,012124,01,1995-09-22,2022-03-31,-0.069002,0000895421,6211,523110,402030,40203020
5,2000-01-31,21186.0,WESTVACO CORP,96154810,11.0,1.0,2631.0,W,011446,01,1957-03-01,2022-03-31,-0.159004,0001159297,2631,322130,151030,15103020
6,2000-01-31,52978.0,HASBRO INC,41805610,11.0,1.0,3944.0,HAS,005518,01,1984-09-13,2022-03-31,-0.204752,0000046080,3944,339930,252020,25202010
7,2000-01-31,21371.0,CARDINAL HEALTH INC,14149Y10,11.0,1.0,5122.0,CAH,002751,01,1997-05-27,2022-03-31,-0.003916,0000721371,5122,424210,351020,35102010
8,2000-01-31,75333.0,BURLINGTON RESOURCES INC,12201410,11.0,1.0,1311.0,BR,015084,01,1993-11-23,2006-03-31,-0.030246,0000833320,1311,211111,101020,10102020
9,2000-01-31,23317.0,ENRON CORP,29356110,11.0,1.0,1311.0,ENE,006127,01,1951-09-06,2001-11-29,0.529577,0001024401,5172,422720,551050,55105010


In [12]:
# Getting a two-year sample of records for 2020 and 2021

sp500_sample = sp500.loc['01/01/2020' <= sp500.date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

sp500_sample = sp500_sample.loc[sp500.date <= '01/01/2022'][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

In [13]:
sp500_sample

,date,permno,comnam,ncusip,gvkey,iid,cik,ticker,sic,naics
120131,2020-01-31,34746.0,FIFTH THIRD BANCORP,31677310,004640,01,0000035527,FITB,6020,522110
120132,2020-01-31,75591.0,IDEX CORP,45167R10,015267,01,0000832101,IEX,3561,333914
120133,2020-01-31,14541.0,CHEVRON CORP NEW,16676410,002991,01,0000093410,CVX,2911,324110
120134,2020-01-31,14593.0,APPLE INC,03783310,001690,01,0000320193,AAPL,3663,334220
120135,2020-01-31,75341.0,DUKE REALTY CORP,26441150,013510,01,0000783280,DRE,6798,531120
...,...,...,...,...,...,...,...,...,...,...
132138,2021-12-31,93096.0,DOLLAR GENERAL CORP NEW,25667710,004016,02,0000029534,DG,5331,452319
132139,2021-12-31,19502.0,WALGREENS BOOTS ALLIANCE INC,93142710,011264,01,0001618921,WBA,5912,446110
132140,2021-12-31,69796.0,CONSTELLATION BRANDS INC,21036P10,002710,02,0000016918,STZ,2082,312120
132141,2021-12-31,62092.0,THERMO FISHER SCIENTIFIC INC,88355610,010530,01,0000097745,TMO,3826,334516


In [7]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# increment the current date by 1 month
def get_next_month(date):
    DATE_FORMAT = "%Y-%m-%d"
    datetime_date = datetime.strptime(date, DATE_FORMAT)
    new_date = datetime_date + relativedelta(months=1)
    return new_date.strftime(DATE_FORMAT)

In [8]:
from sec_edgar_downloader import Downloader

def get_filing(ticker, date, dir):
    num_filings = 0
    dl = Downloader(dir)
    num_filings += dl.get("10-Q", ticker, after=date, before=get_next_month
    (date))
    num_filings += dl.get("10-K", ticker, after=date, before=get_next_month
    (date))
    return num_filings

In [9]:
# this function retrieves all S&P 500 10-Q or 10-K filings for a given month,
# if such a filing exists for a given company
# the date should be passed in the following format: YYYY-MM-DD
def get_all_sp_filings(date, dir):
    num_filings = 0
    sp500_companies = sp500_sample.loc[sp500_sample.date == date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]
    sp500_tickers = sp500_companies['ticker']

    for ticker in sp500_tickers:
        num_filings += get_filing(ticker, date, dir)

    return num_filings

In [ ]:
DATA_FOLDER = "data"

filings = get_all_sp_filings("2020-01-31", DATA_FOLDER)
filings

KeyboardInterrupt: 

In [10]:
# parsing the records
from bs4 import BeautifulSoup

In [11]:
with open("data/sec-edgar-filings/AAPL/10-Q/0000320193-20-000010/filing-details.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [12]:
from bs4 import BeautifulSoup
import os

# this function retrieves all filing company tickers from a given directory
def get_all_filing_tickers(dir_name):
    files = os.listdir(dir_name + "/sec-edgar-filings")
    return files

In [13]:
print(get_all_filing_tickers("data"))

['VZ', 'AMZN', 'CNP', 'RCL', 'CAT', 'AAPL', 'KHC', 'AGN', 'ANET', 'CAH', 'FBHS', 'PFE', 'REG', 'APTV', 'AAL', 'CDW', 'MAR', 'VRSN', 'KMI', 'SPGI', 'ALLE', 'SYK', 'PEP', 'FRT', 'SNPS', 'PLD', 'MMM', 'EMN', 'AMT', 'ADI', 'MAA', 'DRE', 'FTV', 'MGM', 'VLO', 'EMR', 'SYY', 'GILD', 'SNA', 'MS', 'ALXN', 'CVX', 'NCLH', 'UHS', 'MO', 'TXN', 'UNM', 'BK', 'BLL', 'TTWO', 'HCA', 'CDNS', 'HII', 'CTSH', 'MCHP', 'VTR', 'HIG', 'ETFC', 'IPG', 'DISH', 'UNP', 'AMAT', 'LLY', 'NTAP', 'WAB', 'REGN', 'RSG', 'IR', 'IQV', 'LMT', 'CI', 'PPL', 'ANSS', 'EL', 'PSX', 'JNJ', 'QCOM', 'LNT', 'BAC', 'IT', 'SIVB', 'DHI', 'IRM', 'DGX', 'EQR', 'ED', 'DHR', 'HSIC', 'ALB', 'NLOK', 'WM', 'COG', 'RHI', 'TMO', 'ALL', 'BSX', 'FAST', 'ABBV', 'AFL', 'RMD', 'TWTR', 'EFX', 'LYB', 'PNR', 'KEY', 'ATVI', 'CHRW', 'COF', 'PRGO', 'VMC', 'HSY', 'T', 'CHTR', 'TAP', 'MCK', 'AIG', 'EBAY', 'F', 'NTRS', 'O', 'CFG', 'STE', 'PYPL', 'IBM', 'AMD', 'BWA', 'MHK', 'PG', 'INCY', 'XOM', 'USB', 'SO', 'LRCX', 'FOXA', 'DTE', 'BRK', 'ROP', 'AME', 'RJF', 'KMB'

In [14]:
from bs4 import BeautifulSoup

FILING_DETAILS = "filing-details.html"
FULL_SUBMISSION = "full-submission.html"

def get_filings_from_ticker(ticker, dir_name, sec_filings):
    for root, dirs, files in os.walk(dir_name + "/sec-edgar-filings/" + ticker):
        for f in files:
            if FILING_DETAILS in f:
                with open(root + "/" + f) as fp:
                    soup = BeautifulSoup(fp, "html.parser")
                    sec_filings[ticker] = soup.get_text()
    return sec_filings

In [15]:
# TEST CODE
filings = {}
filings = get_filings_from_ticker("AAL", "data", filings)

In [16]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dennisfenchenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

In [19]:
# a function that tokenizes 10-K or 10-Q corpus that
def tokenize_filing(filing_corpus):
    filing_tokenizer = RegexpTokenizer(r'\w+')
    return filing_tokenizer.tokenize(filing_corpus)

In [25]:
# a function to remove the stop words from the filings corpus
def filter_stopwords(tokenized_filing_corpus):
    stop_words = set(stopwords.words('english'))
    filing_corpus_filtered = [word for word in tokenized_filing_corpus if not \
        word.lower() in stop_words]
    return filing_corpus_filtered

def filter_numbers(filing_corpus):
    return [token for token in filing_corpus if not (token.isdigit()
                                         or token[0] == '-' and token[1:].isdigit())]

In [26]:
# TEST CODE
filtered_corpus = filter_stopwords(filings.get("AAL"))
filtered_corpus = filter_numbers(filtered_corpus)
print(len(filings.get("AAL")))
print(len(filtered_corpus))

817948
474072


In [109]:
# get CRSP stock performance for each given company in a given quarter
# we will use the Value-Weighted Return (including distributions) for that
conn.list_tables(library='crsp')

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'cap',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'contact_info',
 'crsp_cik_map',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_portno_map',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'daily_nav',
 'daily_nav_ret',
 'daily_returns',
 'dividends',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',


In [115]:
company = conn.get_table(library='crsp', table='msp500p', obs=5)

In [116]:
company

,caldt,vwretd,vwretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,sprtrn
0,1925-12-31,NaN,NaN,NaN,NaN,15236829.5,89.0,NaN,NaN,12.46,NaN
1,1926-01-30,-0.001783,-0.003980,0.006457,0.003250,15277664.0,89.0,15236829.5,79.0,12.74,0.022472
2,1926-02-27,-0.033296,-0.037876,-0.039979,-0.042451,14712894.9,89.0,15277664.0,81.0,12.18,-0.043956
3,1926-03-31,-0.057708,-0.062007,-0.067915,-0.073275,14012079.2,89.0,14712894.9,81.0,11.46,-0.059113
4,1926-04-30,0.038522,0.034856,0.031441,0.027121,14500482.2,89.0,14012079.2,82.0,11.72,0.022688


In [ ]:
# Get 10Q filing data for each S&P 500 company in a given quarter

In [1]:
# Perform sentiment analysis on filing documents
import pysentiment2 as ps

# get the Harvard general sentiment dictionary
hiv4 = ps.HIV4()

# get the Loughran and McDonald dictionary
lm = ps.LM()